In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import pymongo

In [2]:
file = "data/clean_hockey_data.csv"
nhl_df = pd.read_csv(file)

In [3]:
nhl_df.head()

,Unnamed: 0,Rk,Player,ID,Age,Pos,Tm,GP,G,A,...,HART,Votes,Season,G/60min,A/60min,PTS/60min,HIT/60min,BLK/60min,PIM/60min,S/60min
0,0,1,Connor McDavid,mcdavco01,20,C,EDM,82.0,30,70,...,1,1604,2017,1.038661,2.423543,3.462204,1.177149,1.004039,0.900173,8.690133
1,1,2,Sidney Crosby,crosbsi01,29,C,PIT,75.0,44,45,...,0,1104,2017,1.770624,1.810865,3.581489,3.219316,1.086519,0.965795,10.261569
2,2,3,Patrick Kane,kanepa01,28,RW,CHI,82.0,34,55,...,0,206,2017,1.163056,1.881414,3.044470,0.957811,0.513113,1.094641,9.988597
3,3,4,Nicklas Backstrom,backsni02,29,C,WSH,82.0,23,63,...,0,60,2017,0.921844,2.525050,3.446894,1.803607,1.322645,1.523046,6.492986
4,4,5,Nikita Kucherov,kucheni01,23,RW,TBL,74.0,40,45,...,0,119,2017,1.668985,1.877608,3.546592,1.251739,0.834492,1.585535,10.264256


In [4]:
# Pass in a df and a category, and the function will produce a new df grouped by that category and displaying 
# the average (or median) of each category
def groupby_avg(df, category):
    # Groups the df by a category
    temp_df = df.groupby([category])

    # Calculates the average stats for each group
    avg_goals = temp_df["G/60min"].mean()
    avg_assists = temp_df["A/60min"].mean()
    avg_pts = temp_df["PTS/60min"].mean()
    avg_hits = temp_df["HIT/60min"].mean()
    avg_blocks = temp_df["BLK/60min"].mean()
    avg_s = temp_df["S/60min"].mean()
    avg_shootperc = temp_df["S_percent"].mean()
    avg_pim = temp_df["PIM/60min"].mean()
    med_pim = temp_df["PIM/60min"].median()
    count = temp_df["Player"].count()

    # Builds the new df
    new_df = pd.DataFrame({
        "Avg G/60min" : avg_goals,
        "Avg A/60min" : avg_assists,
        "Avg PTS/60min" : avg_pts,
        "Avg HIT/60min" : avg_hits,
        "Avg BLK/60min" : avg_blocks,
        "Avg S/60min" : avg_s,
        "Avg S_percent" : avg_shootperc,
        "Avg PIM/60min" : avg_pim,
        "Med PIM/60min" : med_pim,
        "Num Players" : count
    })
    return new_df


def groupby_median(df, category):
    temp_df = df.groupby([category])

    med_goals = temp_df["G/60min"].median()
    med_assists = temp_df["A/60min"].median()
    med_pts = temp_df["PTS/60min"].median()
    med_hits = temp_df["HIT/60min"].median()
    med_blocks = temp_df["BLK/60min"].median()
    med_shhotperc = temp_df["S_percent"].median()
    med_pim = temp_df["PIM/60min"].median()
    count = temp_df["Player"].count()


    new_df = pd.DataFrame({
        "Med G/60min" : med_goals,
        "Med A/60min" : med_assists,
        "Med PTS/60min" : med_pts,
        "Med HIT/60min" : med_hits,
        "Med BLK/60min" : med_blocks,
        "Med PIM/60min" : med_pim,
        "Num Players" : count
    })
    return new_df

In [5]:
# Df grouping by 2 categories
def multi_groupby_med(df, category1, category2):
    temp_df = df.groupby([category1, category2])

    med_goals = temp_df["G/60min"].median()
    med_assists = temp_df["A/60min"].median()
    med_pts = temp_df["PTS/60min"].median()
    med_hits = temp_df["HIT/60min"].median()
    med_blocks = temp_df["BLK/60min"].median()
    med_pim = temp_df["PIM/60min"].median()
    count = temp_df["Age"].count()

    new_df = pd.DataFrame({
        "Med G/60min" : med_goals,
        "Med A/60min" : med_assists,
        "Med PTS/60min" : med_pts,
        "Med HIT/60min" : med_hits,
        "Med BLK/60min" : med_blocks,
        "Med PIM/60min" : med_pim,
        "Num Players" : count
    })
    return new_df

def multi_groupby_avg(df, category1, category2):
    temp_df = df.groupby([category1, category2])

    avg_goals = temp_df["G/60min"].mean()
    avg_assists = temp_df["A/60min"].mean()
    avg_pts = temp_df["PTS/60min"].mean()
    avg_hits = temp_df["HIT/60min"].mean()
    avg_blocks = temp_df["BLK/60min"].mean()
    avg_pim = temp_df["PIM/60min"].mean()
    med_pim = temp_df["PIM/60min"].median()
    avg_s = temp_df["S/60min"].mean()
    count = temp_df["Age"].count()

    new_df = pd.DataFrame({
        "Avg G/60min" : avg_goals,
        "Avg A/60min" : avg_assists,
        "Avg PTS/60min" : avg_pts,
        "Avg HIT/60min" : avg_hits,
        "Avg BLK/60min" : avg_blocks,
        "Avg PIM/60min" : avg_pim,
        "Med PIM/60min" : med_pim,
        "Avg S/60min" : avg_s,
        "Avg Players" : count
    })
    return new_df

In [6]:
def seasonyears_filter(df, years_list):
    new_df = df[df.Season.isin(years_list)]
    return new_df

# Groups a df into 3 sets of 3 years
def three_year_group(df):
    years = [2004, 2005, 2006, 2009, 2010, 2011, 2016, 2017, 2018]
    new_df = df[df.Season.isin(years)]

    bins = [2004, 2006, 2011, 2018]
    groups = ["2004-06", "2009-11", "2016-18"]
    new_df["Season_group"] = pd.cut(new_df["Season"], bins, labels=groups, include_lowest=True)
    # grouped_df = multi_groupby_avg(new_df, "Age", "Season_group")
    return new_df

# dfs to be moved to database

In [7]:
complete_nhl_df = nhl_df
complete_nhl_df.head()

,Unnamed: 0,Rk,Player,ID,Age,Pos,Tm,GP,G,A,...,HART,Votes,Season,G/60min,A/60min,PTS/60min,HIT/60min,BLK/60min,PIM/60min,S/60min
0,0,1,Connor McDavid,mcdavco01,20,C,EDM,82.0,30,70,...,1,1604,2017,1.038661,2.423543,3.462204,1.177149,1.004039,0.900173,8.690133
1,1,2,Sidney Crosby,crosbsi01,29,C,PIT,75.0,44,45,...,0,1104,2017,1.770624,1.810865,3.581489,3.219316,1.086519,0.965795,10.261569
2,2,3,Patrick Kane,kanepa01,28,RW,CHI,82.0,34,55,...,0,206,2017,1.163056,1.881414,3.044470,0.957811,0.513113,1.094641,9.988597
3,3,4,Nicklas Backstrom,backsni02,29,C,WSH,82.0,23,63,...,0,60,2017,0.921844,2.525050,3.446894,1.803607,1.322645,1.523046,6.492986
4,4,5,Nikita Kucherov,kucheni01,23,RW,TBL,74.0,40,45,...,0,119,2017,1.668985,1.877608,3.546592,1.251739,0.834492,1.585535,10.264256


In [8]:
age_df = groupby_avg(complete_nhl_df, "Age")
age_df.head()

,Avg G/60min,Avg A/60min,Avg PTS/60min,Avg HIT/60min,Avg BLK/60min,Avg S/60min,Avg S_percent,Avg PIM/60min,Med PIM/60min,Num Players
Age,,,,,,,,,,
18,0.824067,1.148905,1.972972,2.490490,1.401981,7.343056,10.941667,1.739567,1.191372,24
19,0.616434,1.000604,1.617038,3.519958,1.890612,6.572784,8.770732,1.736700,1.368243,123
20,0.673296,0.995643,1.668939,3.063742,1.597840,6.738494,9.348246,1.977292,1.416767,228
21,0.613062,0.951827,1.564889,3.575405,1.734297,6.527488,8.644836,2.310811,1.665270,426
22,0.589820,0.928056,1.517876,3.951092,1.929304,6.426348,8.447611,2.505728,1.701871,565


In [9]:
season_df = groupby_avg(complete_nhl_df, "Season")
season_df.head()

,Avg G/60min,Avg A/60min,Avg PTS/60min,Avg HIT/60min,Avg BLK/60min,Avg S/60min,Avg S_percent,Avg PIM/60min,Med PIM/60min,Num Players
Season,,,,,,,,,,
2004,0.514478,0.828224,1.342702,0.000000,0.000000,5.705203,8.308504,4.328171,2.272317,635
2006,0.609734,1.009904,1.619638,0.000000,0.000000,6.174615,8.911111,4.131063,2.882364,630
2007,0.576399,0.966838,1.543237,0.000000,0.000000,6.090347,8.576645,3.631224,2.395814,608
2008,0.534129,0.888737,1.422866,4.500742,2.339818,5.935895,8.137239,3.691121,2.148876,623
2009,0.569995,0.951735,1.521730,2.477101,1.288872,6.098845,8.465959,3.611024,2.240199,1225


# Loading into MongoDB

In [12]:
conn ='mongodb://localhost:27017' 

In [13]:
client = pymongo.MongoClient(conn)

In [27]:
client.drop_database('nhl_db')

In [28]:
db = client.nhl_db

In [29]:
collection = db.nhl_player_data

In [30]:
complete_nhl_df.columns

Index(['Unnamed: 0', 'Rk', 'Player', 'ID', 'Age', 'Pos', 'Tm', 'GP', 'G', 'A',
       'PTS', 'plusminus', 'PIM', 'PS', 'EVG', 'PPG', 'SHG', 'GW', 'EVA',
       'PPA', 'SHA', 'S', 'S_percent', 'TOI', 'ATOI', 'BLK', 'HIT', 'FOW',
       'FOL', 'FO_percent', 'HART', 'Votes', 'Season', 'G/60min', 'A/60min',
       'PTS/60min', 'HIT/60min', 'BLK/60min', 'PIM/60min'],
      dtype='object')

In [31]:
data = complete_nhl_df.to_dict(orient='records')
collection.insert_many(data)

In [32]:
nhl_col=db.nhl_player_data.find()
for col in nhl_col:
    print(col)

 0, 'S': 38, 'S_percent': 2.6, 'TOI': 668, 'ATOI': 18.1, 'BLK': 72.0, 'HIT': 33, 'FOW': 0.0, 'FOL': 0.0, 'FO_percent': nan, 'HART': 0, 'Votes': 0, 'Season': 2018, 'G/60min': 0.08982035928143713, 'A/60min': 0.3592814371257485, 'PTS/60min': 0.4491017964071856, 'HIT/60min': 2.9640718562874246, 'BLK/60min': 6.4670658682634725, 'PIM/60min': 0.5389221556886228}
{'_id': ObjectId('5efe99b91db134b130c3b367'), 'Unnamed: 0': 12951, 'Rk': 656, 'Player': 'Dennis Seidenberg', 'ID': 'seidede01', 'Age': 36, 'Pos': 'D', 'Tm': 'NYI', 'GP': 28.0, 'G': 0, 'A': 5, 'PTS': 5, 'plusminus': -9, 'PIM': 17, 'PS': 0.0, 'EVG': 0, 'PPG': 0, 'SHG': 0, 'GW': 0, 'EVA': 5, 'PPA': 0, 'SHA': 0, 'S': 31, 'S_percent': 0.0, 'TOI': 501, 'ATOI': 17.9, 'BLK': 43.0, 'HIT': 65, 'FOW': 0.0, 'FOL': 0.0, 'FO_percent': nan, 'HART': 0, 'Votes': 0, 'Season': 2018, 'G/60min': 0.0, 'A/60min': 0.5988023952095809, 'PTS/60min': 0.5988023952095809, 'HIT/60min': 7.78443113772455, 'BLK/60min': 5.1497005988023945, 'PIM/60min': 2.03592814371257

In [33]:
collection2 = db.age_groups

In [34]:
data2 = age_df.to_dict(orient='records')
collection2.insert_many(data2)

In [35]:
collection3 = db.season_groups

In [36]:
data3 = season_df.to_dict(orient='records')
collection3.insert_many(data3)